## Factors Influencing HIT ##

**BETA RELEASE**

*This project remains in the experimental stage and there are likely many tweaks to come that will improve/change outcomes. I do believe at this stage that the simulation provides good general insights regarding the relationship between various factors and viral spread.*

**REPLICABILITY**

*Randomness is in an important factor in RKnot's approach to simulation (and frankly in real-word viral transmission), so the results of the sims below will not be repeatable with each iteration. The below examples are meant to show general differences based on state; further analysis should run the same simulation multiple times to see the mean impact.*

**SARS-COV-2**

*While sars-cov-2 is obviously topical globally, the goal of RKnot is to be applicable to any virus (or anything that spreads like a virus). sars-cov-2 is used only for illustration purposes.*

### Base US Simulation ###

To explore the various concepts of RKnot and viral spread, we'll use a simulation design based on [CDC Best Planning Scenario](https://www.cdc.gov/coronavirus/2019-ncov/hcp/planning-scenarios.html) guidelines for
COVID-19 characteristics including:

* $R_0$ 2.5
* IFR for each of 4 age groups
    * 0-19: 0.003%
    * 20-49: 0.02%
    * 50-69: 0.5%
    * 70+: 5.4%

Other assumptions:

* Population of $10,000^1$
* Initial Infected of 2
* Duration of Infectiousness 14 days$^2$
* Duration of Immunity 365 days
* Density of ~1 subject per location (`dlevel='med'`)

$^1$*proportionately split among the 4 age groups to match US census data.*

$^2$*equal likelihood of transmission on any day (i.e. no viral load curve)*

[US Census Data](https://www.census.gov/prod/cen2010/briefs/c2010br-03.pdf)

### Natural ###

#### 1. Equal ####
The first simulation makes the most homogeneous assumptions. 

+ No group is restricted in terms of movement. 
+ All dots are able to interact with one another. 
+ All dots are susceptible at initiation.
+ All dots are equally likely to move to any dot on the grid (`mover='equal'`)

In [1]:
### HIDDEN ###
import matplotlib
matplotlib.use('Qt5Agg')

from matplotlib.animation import FFMpegWriter

In [2]:
### HIDDEN ###
%load_ext autoreload
%autoreload 2
RUN = True
SHOW_CHARTS = False
SAVE_CHARTS = True
PATH = 'vids/baseus/'

The basic layout is below. These parameters can also be imported from `rknot.sims.baseus` for convenienve.

```python
group1 = dict(
    name='0-19',
    n=2700,
    n_inf=0,
    ifr=0.00003,
    mover='equal',
)
group2 = dict(
    name='20-49',
    n=4100,
    n_inf=1,
    ifr=0.0002,
    mover='equal',
)
group3 = dict(
    name='50-69',
    n=2300,
    n_inf=1,
    ifr=0.005,
    mover='equal',
)
group4 = dict(
    name='70+',
    n=900,
    n_inf=0,
    ifr=0.054,
    mover='equal',
)
groups = [group1, group2, group3, group4]
params = {'dlevel': 'med', 'Ro':2.5, 'days': 365, 'imndur': 365, 'infdur': 14}
```

In [3]:
### HIDDEN ###
from rknot.sims.baseus import params, groups
from rknot import Sim, Chart
# sim = Sim(groups=groups, details=True, **params)
for group in groups:
    group['mover'] = 'equal'

```python
from rknot import Sim, Chart
```

In [5]:
sim = Sim(groups=groups, details=True, **params)

2020-10-16 23:17:22,257	INFO resource_spec.py:223 -- Starting Ray with 12.01 GiB memory available for workers and up to 6.02 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-10-16 23:17:22,701	INFO services.py:1191 -- View the Ray dashboard at localhost:8265


---------------------------------------------------------------------------------
|                                  SIM DETAILS                                  |
|-------------------------------------------------------------------------------|
|           boundary|  [  1 100   1 100]|             n_locs|              10000|
|-------------------|-------------------|-------------------|-------------------|
|                  n|              10000|            density|                1.0|
|-------------------|-------------------|-------------------|-------------------|
|                ktr|                1.0|                   |                   |
|-------------------|-------------------|-------------------|-------------------|


Running the animation will result in a video as per below:

```python
sim.run()
chart = Chart(sim, use_init_func=True)
chart.animate.to_html5_video()
```

In [6]:
if RUN:
    sim.run()
    chart = Chart(sim, use_init_func=True)
if SAVE_CHARTS:
    chart.animate.save(PATH + '/1.mp4', writer=FFMpegWriter(fps=10))

<video width='705' height='340' 
    src='https://storage.googleapis.com/rknotvids/baseus/1.mp4' controls>alternative text</video>

*Note embedded videos are used for convenience purposes. Given the random processes involved, running the same code will produce slightly different results each time.*

The sim results in a relatively standard spread curve with 

|         |            |
| ------------- |:-------------:|
| Peak      |   41% |
| HIT      | 65%      |
| Fatalities | 0.52%      |

You'll note this is slightly higher than the expected HIT of 60% for $R_0$ 2.5. The differences result given:

1. The simulation is a stochastic process and random variation will result in variation from the mathematical model in smaller populations.
2. This is not an entirely homogeneous population, with different IFRs and different numbers of contacts between subjects.

#### 2. Local ####
In remaining simulations, we begin to introduce ever increasing homogeneity.

Our first change is to adjust the subjects mover functions to `local`. The `local` mover has a strong bias towards locations only in its immediate vicinity, which is a better approximation of real world processes (though certainly not a perfect one).

```python
group1['mover'] = 'local'
group2['mover'] = 'local'
group3['mover'] = 'local'
group4['mover'] = 'local'
groups = [group1, group2, group3, group4]

sim = Sim(groups=groups, **params)
sim.run()

chart = Chart(sim, use_init_func=True)
chart.animate.to_html5_video()
```

In [2]:
### HIDDEN ###
if RUN:
    from rknot.sims.baseus import params, groups
    from rknot import Sim, Chart
    for group in groups:
        group['mover'] = 'local'

    sim = Sim(groups=groups, **params)
    sim.run()

if SAVE_CHARTS:
    chart = Chart(sim, use_init_func=True)
    chart.animate.save(PATH + '2.mp4', writer=FFMpegWriter(fps=10))

NameError: name 'PATH' is not defined

<video width='705' height='340' 
    src='https://storage.googleapis.com/rknotvids/baseus/2.mp4' controls>Video Failed To Load</video>

This results in:

|         |            |
| ------------- |:-------------:|
| Peak      |   5% |
| HIT      | 49%      |
| Fatalities | 0.38%      |

We can see that restricting movement has a major impact on spread, creating a much longer, flatter curve with very low peak infection and a lower HIT than $R_0$ would suggest. Fatalities have also reduced by ~25% from [Example 1](#Equal).

The virus does remain among the population, however, resulting in high penetration and relatively high fatalities. For comparison a 0.38% fatality rate is equivalent to >1MM deaths across the US.

So while HIT is an important consideration, it should not be the sole focus if the ultimate goal is to reduce fatalities.

*CAUTION: RKnot estimates transmission rate assuming all subjects are* `equal` *movers. Thus, where* `local` *movers are used, the transmission rate may (or may not) be underestimated.*

#### 3. Social  ####
In this simulation, we set `mover=social` for just the 20-49 age group. This is a rough approximation of that group's real-world propensity to travel more frequently (or go to more events).

```python
group1['mover'] = 'local'
group2['mover'] = 'social'
group3['mover'] = 'local'
group4['mover'] = 'local'
groups = [group1, group2, group3, group4]

sim = Sim(groups=groups, **params)
sim.run()

chart = Chart(sim, use_init_func=True)
chart.animate.to_html5_video()
```

In [8]:
if RUN:
    groups[1]['mover'] = 'social'
    
    sim = Sim(groups=groups, **params)
    sim.run()

    chart = Chart(sim, use_init_func=True)
if SAVE_CHARTS:
    chart.animate.save('vids/baseus/3.mp4', writer=FFMpegWriter(fps=10))

<video width='705' height='340' 
    src='https://storage.googleapis.com/rknotvids/baseus/3.mp4' controls>Video Failed To Load</video>

This results in:

|         |            |
| ------------- |:-------------:|
| Peak      |   34% |
| HIT      | 62%      |
| Fatalities | 0.61%      |

We can see how powerful mixing is within a population. Even with the majority of subjects moving only locally, if a small group of subjects is moving more broadly across the space, it will significantly increase the rate of spread.

#### 4. Pre-Existing Immunity ####
In this scenario, we adjust the susceptibility factor for just two groups by relatively small amounts as follows:

+ 20-49: 80%
+ 50-69: 65%

This means, in the inverse, that 10% and 25% of the subjects in the respective groups are already immune to the virus (whether through pre-existing T-cell immunity, anti-bodies, or otherwise).

The older group is assumed to have a lower susceptibility factor as it is more likely that older people will have had more exposure to similar viruses over their lifetime.

T-cell immunity to sars-cov-2 remains a controversial subject, but [many studies](https://www.bmj.com/content/370/bmj.m3563) have found prevalance of T-cells between 20% - 50% in *people unexposed to sars-cov-2*. It is suggested that exposure to "common cold" coronaviruses (or more dangerous ones) may convey this immunity.

```python
group2['susf'] = .8
group3['susf'] = .65
groups = [group1, group2, group3, group4]

sim = Sim(groups=groups, **params)
sim.run()

chart = Chart(sim, use_init_func=True) 
chart.animate.to_html5_video()
```

In [9]:
if RUN:
    groups[1]['susf'] = .8
    groups[2]['susf'] = .65

    sim = Sim(groups=groups, **params)
    sim.run()

    chart = Chart(sim, use_init_func=True) 
if SAVE_CHARTS:
    chart.animate.save('vids/baseus/4.mp4', writer=FFMpegWriter(fps=10))

<video width='705' height='340'
    src='https://storage.googleapis.com/rknotvids/baseus/4.mp4' controls>Video Failed to Load</video>

This results in:

|         |            |
| ------------- |:-------------:|
| Peak      |   21% |
| HIT      | 44%      |
| Fatalities | 0.49%      |

We can see that pre-existing immunity would significantly reduce HIT ... but again reducing HIT does not impact the fatality rate as much as one might expect.

Comparing to Example [3](#Social), HIT reduced by a third and fatalities declined by only a sixth.

In this sim specifically, this results at least in part from the fact the elderly are assumed to *not* have pre-existing immunity.

#### 5. Events ####
Certainly, the vast majority of people in the US do not move in such pre-defined ways as set out by the `mover` function. In reality, people tend to move with a local bias with a small number of interactions, supplemented by larger movements to locations with a large number of interactions in a small amount of time.

In RKnot, we can simulate this with [Event objects](concepts.ipynb#Events). And we will incorporate a number of them in this simulation. 

First, we will reset our group parameters by importing from `sims.baseus`.

Then we will instantiate a handful of events recurring periodically over the duration of the sim.

```python
from rknot.sims.baseus import params, groups

from rknot.events import Event

school1 = Event(name='school1', xy=[25,42], start_tick=2, groups=[0], capacity=10, recurring=2)
school2 = Event(name='school2', xy=[78,82], start_tick=3, groups=[0], capacity=10, recurring=2)
game = Event(name='game', xy=[50,84], start_tick=6, groups=[0,1,2], capacity=100, recurring=14)
concert1 = Event(name='concert1', xy=[20,20], start_tick=7, groups=[1], capacity=50, recurring=14)                 
concert2 = Event(name='concert2', xy=[91,92], start_tick=21, groups=[1], capacity=50, recurring=14)
bar = Event(name='bar', xy=[17,24], start_tick=4, groups=[1], capacity=5, recurring=7)
bar2 = Event(name='bar2', xy=[87,13], start_tick=5, groups=[1], capacity=5, recurring=7)
bar3 = Event(name='bar3', xy=[52,89], start_tick=6, groups=[1,2], capacity=5, recurring=7)
church = Event(name='church', xy=[2,91], start_tick=7, groups=[2,3], capacity=20, recurring=7)

events = [school1, school2, game, concert1, concert2, bar, bar2, bar3, church]

sim = Sim(groups=groups, events=events, **params)
sim.run()

chart = Chart(sim, use_init_func=True)
chart.animate.to_html5_video()
```

In [2]:
if RUN:
    from rknot import Sim, Chart
    from rknot.sims.baseus import params, groups, events

    sim = Sim(groups=groups, events=events, **params)
    sim.run()


if SAVE_CHARTS:
    chart = Chart(sim, use_init_func=True)
    chart.animate.save(PATH + '5.mp4', writer=FFMpegWriter(fps=10))

<video width='705' height='340'
    src='https://storage.googleapis.com/rknotvids/baseus/5.mp4' controls>Video Failed to Load</video>

This results in:

|         |            |
| ------------- |:-------------:|
| Peak      |   22% |
| HIT      | 52%      |
| Fatalities | 0.48%      |

We can see that this sort of event-based structure resulted in lower peak infections and HIT relative to [Example 1](#Equal) & [Example 3](#Social). The curve is flatter and longer. 

This scenario are not perfectly substitutable with [Example 1](#Equal). For example, lower spread might result because there are fewer contacts. BUT this scenario actually results in more total interactions between subjects than the prior ones:

Avg Number of Contacts per Subject:

Example 1: 50.3

Example 2: 51.7 

Example 3: 53.8

Example 5: 56.5

*Average of first 100 days across 5 sims for each scenario.*

Other issues with substitutability may exist and are being explored. SIR models determine R0 in the idealized environment of Example 1 and so may not be suitable for customized environments such as this one.


#### 6. Gates ####
We can further improve the real world relevance of interactions by introducing [gates](concepts.ipynb#Gates). Subjects are not always freely able to interact with all other people in a population. Often their movement is restricted to within certain areas. Furthermore, other people's access into those areas is restricted.

Elderly people living in retirement homes or assisted living centers is an example. To simulate this, we will split `group4` into two separate groups. 

* `group4a`
    * population of 600 (2/3s of `group4`) 
    * IFR of 4.2%
    * move freely throughout the entire grid as previously
* `group4b` 
    * population of 300 (1/3rd of `group4`)
    * IFR of 7.8%
    * movement restricted to 6x6 box

We have also adjusted the IFR on the basis that `group4b` is likely older and also probably more frail than `group4a`. IFRs approximate those found [here](https://www.publichealthontario.ca/-/media/documents/ncov/epi/2020/06/covid19-epi-case-identification-age-only-template.pdf?la=en).

In addition, we will update certain events to account for the additional group including adding an event specifically for the new group.

```python
group4a = dict(
    name='70+',
    n=600,
    n_inf=0,
    ifr=0.042,
    mover='local',
)
group4b = dict(
    name='70+G',
    n=300,
    n_inf=0,
    ifr=0.0683,
    mover='local',
    box=[1,6,1,6],
    box_is_gated=True,
)
groups = [group1, group2, group3, group4a, group4b]

game = Event(name='game', xy=[50,84], start_tick=6, groups=[0,1,2,3], capacity=100, recurring=14)
church = Event(name='church', xy=[2,91], start_tick=7, groups=[2,3], capacity=20, recurring=7)
church2 = Event(name='church', xy=[2,3], start_tick=7, groups=[4], capacity=5, recurring=7)

events = [game, concert1, concert2, bar, bar2, bar3, church, church2]
```

Now, such elderly populations are not totally sealed of from the rest of the world. In fact, they are often visited by family members or friends. We can mimick this with the use of a Travel object.

In this sim, at least one person will enter into the `group4b` gate for a day only. And this will repeat every day of the sim.

```python
from rknot.events import Travel

visit = Travel(name='visit', xy=[1,1], start_tick=3, groups=[1,2], capacity=1, duration=1, recurring=1)
events.append(visit)

sim = Sim(groups=groups, events=events, **params)
sim.run()

chart = Chart(sim, use_init_func=True)
chart.animate.to_html5_video()
```

In [4]:
if RUN:
    from rknot import Sim, Chart
    from rknot.sims.baseus import params, events_gated, groups_gated
    
    sim = Sim(groups=groups_gated, events=events_gated, **params)
    sim.run()

if SAVE_CHARTS:
    chart = Chart(sim, use_init_func=True)    
    chart.animate.save(PATH + '6.mp4', writer=FFMpegWriter(fps=10))

<video width='705' height='340'
    src='https://storage.googleapis.com/rknotvids/baseus/6.mp4' controls>Video Failed to Load</video>

This results in:

|         |            |
| ------------- |:-------------:|
| Peak      |   26% |
| HIT      | 51%      |
| Fatalities | 0.58%      |

This gated structure results in a similar curve to [Example 5](#Events), however, it does result in more fatalities as an outbreak occurs in the most susceptible 70+G group inside the gate. The use for this structure will become more evident in the disucssion on [Policy Decisions.](#Policy)

#### 7. Events, Gates, and Pre-Immunity ####
Now we can see how pre-immunity might impact viral spread in a population with more heterogeneous interactions.

```python
group2['susf'] = .8
group3['susf'] = .65
groups = [group1, group2, group3, group4a, group4b]

sim = Sim(groups=groups, events=events, **params)
sim.run()

chart = Chart(sim, use_init_func=True)
chart.animate.to_html5_video()
```

In [5]:
if RUN:
    from rknot import Sim, Chart
    from rknot.sims.baseus import groups_gated, events_gated, params
    groups_gated[1]['susf'] = .8
    groups_gated[2]['susf'] = .65

    sim = Sim(groups=groups_gated, events=events_gated, **params)
    sim.run()

if SAVE_CHARTS:
    chart = Chart(sim, use_init_func=True)
    chart.animate.save(PATH + 'tests/7.mp4', writer=FFMpegWriter(fps=10))

<video width='705' height='340'
    src='https://storage.googleapis.com/rknotvids/baseus/7.mp4' controls>Video Failed to Load</video>

This results in:

|         |            |
| ------------- |:-------------:|
| Peak      |   14% |
| HIT      | 22%      |
| Fatalities | 0.57%      |

So once again we see that pre-existing immunity would have the effect of reducing peak infections and HIT.

Still, fatalities were not impacted because of an outbreak in '70+G' age group (which you can see from the full grey square in the bottom left hand corner of the grid) ... this despite the gated area and limiting travel to one visitor a day.

#### 8. Self Aware Social Distancing ####
In a self-aware population, we can also incorporate an assumption that certain members of the population will implement social distancing practices (even in the absence of prescribed government policy). For example, individuals might wear masks or face shields while in public. 

This is implemented via a SocialDistancing object, which reduces the transmission factor of the subjects in the applicable group.

```python
from rknot.events import SocialDistancing as SD

sd = SD(name='6-feet', tmfs=[.975,.95,.75,.5], groups=[1,2,3,4], start_tick=5, duration=90)
events.append(sd)

sim = Sim(groups=groups, events=events, **params)
sim.run()

chart = Chart(sim, use_init_func=True)
chart.animate.to_html5_video()
```

In [13]:
if RUN:
    from rknot.events import SocialDistancing as SD

    sd = SD(name='6-feet', tmfs=[.975,.95,.75,.5], groups=[1,2,3,4], start_tick=5, duration=90)
    events_gated.append(sd)

    sim = Sim(groups=groups_gated, events=events_gated, **params)
    sim.run()

    chart = Chart(sim, use_init_func=True)
if SAVE_CHARTS:
    chart.animate.save('vids/baseus/8.mp4', writer=FFMpegWriter(fps=10))

<video width='705' height='340'
    src='https://storage.googleapis.com/rknotvids/baseus/8.mp4' controls>Video Failed to Load</video>

This results in:

|         |            |
| ------------- |:-------------:|
| Peak      |   18% |
| HIT      | 35%      |
| Fatalities | 0.38%      |

Here we can see that social distancing in the broader population had almost no impact on peak infections or HIT.

**BUT** social distancing among the most susceptible (i.e. `group4b`) was a key factor in a dramatically reduced fatality rate.

Essentially, in this simulation, social distancing prevented an outbreak in the `group4b` community (whereas outbreaks had previously occured there in prior sims).

#### 9. Isolation ####
TBD

#### 10. Infectiousness Curve ####
TBD

### Policy ###

With a more realistic model of subject interaction, we can begin to experiment with the impact of different policy measures.

RKnot can simulate policy measures via `Restriction`, `SocialDistancing`, and `Quarantine` objects. Further details [here](concepts.ipynb#Restrictions).

The simulations are based on [this scenario](#Gates) and so the impact of the policy measures contemplated should be considered relative to that scenario.

The structure can be imported as follows:

```python
from rknot.sims.baseus import params, events_gated, groups_gated
```

In [1]:
### HIDDEN ###
import matplotlib
matplotlib.use('Qt5Agg')

from matplotlib.animation import FFMpegWriter

from rknot import Sim, Chart
from rknot.sims.baseus import params, events_gated, groups_gated

#### 1. Restrict Large Gatherings ####
We'll start by simply restricting large gathers, which for this sim is any event with 10+ capacity (0.1% of the entire population). The restrictions will last for 60 days.

When considering capacity, remember that a 100,000-seat stadium in a 10 million person catchment represents 1% of the population.

We assume this policy are implemented on day 30, when the population finally realizes there is a pandemic and government has had time to implement prevention measures.

The restriction will last for 60 days.

```python
from rknot.events import Restriction

large_gatherings = Restriction(
    name='large', start_tick=30, duration=60, criteria={'capacity': 10}
)
events_w_res = events_gated + [large_gatherings]

sim = Sim(groups=groups_gated, events=events_w_res, details=True, **params)
sim.run()

chart = Chart(sim, use_init_func=True)
chart.animate.to_html5_video()
```

In [5]:
if RUN:
    from rknot.events import Restriction

    large_gatherings = Restriction(
        name='large', start_tick=30, duration=60, criteria={'capacity': 10}
    )
    events_w_res = events_gated + [large_gatherings]

    sim = Sim(groups=groups_gated, events=events_w_res, details=True, **params)
    sim.run()

    chart = Chart(sim, use_init_func=True)

if SAVE_CHARTS:
    chart.animate.save('vids/baseus/large.mp4', writer=FFMpegWriter(fps=10))

2020-10-17 00:09:38,928	INFO resource_spec.py:223 -- Starting Ray with 10.01 GiB memory available for workers and up to 5.01 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-10-17 00:09:39,370	INFO services.py:1191 -- View the Ray dashboard at localhost:8265


---------------------------------------------------------------------------------
|                                  SIM DETAILS                                  |
|-------------------------------------------------------------------------------|
|           boundary|  [  1 100   1 100]|             n_locs|              10000|
|-------------------|-------------------|-------------------|-------------------|
|                  n|              10000|            density|                1.0|
|-------------------|-------------------|-------------------|-------------------|
|                ktr|                1.0|                   |                   |
|-------------------|-------------------|-------------------|-------------------|


<video width='705' height='340'
    src='https://storage.googleapis.com/rknotvids/baseus/large.mp4' controls>Video Failed to Load</video>


This results in:

|         |            |
| ------------- |:-------------:|
| Peak      |   19% |
| HIT      | 48%      |
| Fatalities | 0.38%      |

We see that the curve is flattened significanlty during the restriction period and the peak of infections has been pushed way out to 100+ days. As the restriction is lifted, however, the outbreak ensues. Notice that the 70+G region was spared from the outbreak until ~100 days, when a wave of infections quickly increased the fatality rate.

#### 2. Restrict Elderly Visits ####
We can restrict events by name by passing `name` key to `criteria`. We can do this to restrict the travel event to the `70+G` area. 

The policy measure is implemented on day 30 and maintained for another 120 days. There will be no other restrictions.

```python
no_visits = Restriction(
    name='no_visits', start_tick=30, duration=120, criteria={'name': 'visit'}
)
events_w_res = events_gated + [no_visits]

sim = Sim(groups=groups_gated, events=events_w_res, **params)
sim.run()

chart = Chart(sim, use_init_func=True)
chart.animate.to_html5_video()
```

In [6]:
if RUN:
    from rknot.events import Restriction    
    no_visits = Restriction(
        name='no_visits', start_tick=30, duration=120, criteria={'name': 'visit'}
    )
    events_w_res = events_gated + [no_visits]

    sim = Sim(groups=groups_gated, events=events_w_res, **params)
    sim.run()

    chart = Chart(sim, use_init_func=True)
if SAVE_CHARTS:
    chart.animate.save('vids/baseus/no_visits.mp4', writer=FFMpegWriter(fps=10))

2020-10-17 00:14:24,870	INFO resource_spec.py:223 -- Starting Ray with 10.3 GiB memory available for workers and up to 5.15 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-10-17 00:14:25,273	INFO services.py:1191 -- View the Ray dashboard at localhost:8265


<video width='705' height='340' 
    src='https://storage.googleapis.com/rknotvids/baseus/no_visits.mp4' controls>Video Failed to Load</video>

This results in:

|         |            |
| ------------- |:-------------:|
| Peak      |   23% |
| HIT      | 46%      |
| Fatalities | 0.33%      |

Interestingly despite having a higher peak and HIT level than prior examples, we see that *fatalities are the lowest of any sim run to this point*. The most susceptible are protected during the peak and the virus has "burned out" fast enough that they are not infected when visits resume.

We can see here that allowing a *high level of infection* among the **least susceptible** has resulted in a *low level of fatalities* among the **most susceptible**.

#### 3. Social Distancing ####
We can mimick the implementation of Social Distancing policies in certain settings via the `SocialDistancing` object. Mask wearing, hand sanitizing, and 6-foot perimeters all provide varying levels of protection.

It is hard to estimate the degree of protection from each, and even harder in combination. For instance, [this study](https://pubmed.ncbi.nlm.nih.gov/23498357/) found anywhere between a 1.1- and 55-fold reduction in exposure to influenza with varying mask designs.

So we provide `tmfs` here for illustration purposes only, attempting to catch all social distancing practices. We also provide different `tmfs` for the different age groups, meant to simulate adherence to policy.

The policy measure is implemented on day 30 and maintained for 120 days.

```python
from rknot.events import SocialDistancing as SD

sd = SD(name='all', tmfs=[.8,.8,.7,.65,.5], groups=[0,1,2,3,4], start_tick=30, duration=120)
events_w_res = events_gated + [sd]

sim = Sim(groups=groups_gated, events=events_w_res, **params)
sim.run()

chart = Chart(sim, use_init_func=True)
chart.animate.to_html5_video()
```

In [7]:
if RUN:
    from rknot.events import SocialDistancing as SD

    sd = SD(name='all', tmfs=[.8,.8,.7,.65,.5], groups=[0,1,2,3,4], start_tick=30, duration=120)
    events_w_res = events_gated + [sd]

    sim = Sim(groups=groups_gated, events=events_w_res, **params)
    sim.run()

    chart = Chart(sim, use_init_func=True)
if SAVE_CHARTS:
    chart.animate.save('vids/baseus/policy_sd.mp4', writer=FFMpegWriter(fps=10))

2020-10-17 00:19:03,697	INFO resource_spec.py:223 -- Starting Ray with 10.11 GiB memory available for workers and up to 5.07 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-10-17 00:19:04,112	INFO services.py:1191 -- View the Ray dashboard at localhost:8265


<video width='705' height='340'
       src='https://storage.googleapis.com/rknotvids/baseus/policy_sd.mp4' controls>Video Failed to Load</video>

This results in:

|         |            |
| ------------- |:-------------:|
| Peak      |   19% |
| HIT      | 63%      |
| Fatalities | 0.58%      |

We can see that social distancing is certainly the most effective approach in terms of "flattening the curve" with the infection peak pushed all the way out to 190+ days. 

Yet again, though, we see infections dramatically rise after the social distancing restrictions are lifted (180 days). 

In addition, there was an outbreak in the 70+G area, which resulted in a much higher fatality rate than other approaches.

#### 4. Social Distancing and Restrict Elderly Visits and Large Gatherings ####
What happens if we combine the three approaches above?

```python
large_gatherings = Restriction(
    name='large', start_tick=30, duration=60, criteria={'capacity': 10}
)
no_visits = Restriction(
    name='no_visits', start_tick=30, duration=120, criteria={'name': 'visit'}
)
sd = SD(name='all', tmfs=[.8,.8,.7,.65,.5], groups=[0,1,2,3,4], start_tick=30, duration=120)
events_w_res = events_gated + [large_gatherings, no_visits, sd]

sim = Sim(groups=groups_gated, events=events_w_res, **params)
sim.run()

chart = Chart(sim, use_init_func=True)
chart.animate.to_html5_video()
```

In [8]:
if RUN:
    large_gatherings = Restriction(
        name='large', start_tick=30, duration=60, criteria={'capacity': 10}
    )
    no_visits = Restriction(
        name='no_visits', start_tick=30, duration=120, criteria={'name': 'visit'}
    )
    sd = SD(name='all', tmfs=[.8,.8,.7,.65,.5], groups=[0,1,2,3,4], start_tick=30, duration=120)
    events_w_res = events_gated + [large_gatherings, no_visits, sd]

    sim = Sim(groups=groups_gated, events=events_w_res, **params)
    sim.run()

    chart = Chart(sim, use_init_func=True)
if SAVE_CHARTS:
    chart.animate.save('vids/baseus/sd_&_no_large_&_no_visits.mp4', writer=FFMpegWriter(fps=10))

2020-10-17 00:26:12,830	INFO resource_spec.py:223 -- Starting Ray with 10.3 GiB memory available for workers and up to 5.16 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-10-17 00:26:13,263	INFO services.py:1191 -- View the Ray dashboard at localhost:8265


<video width='705' height='340'
    src='https://storage.googleapis.com/rknotvids/baseus/sd_%26_no_large_%26_no_visits.mp4' controls>Video Failed to Load</video>

This results in:

|         |            |
| ------------- |:-------------:|
| Peak      |   12% |
| HIT      | 60%      |
| Fatalities | 0.48%      |

We can see that this combined approach results in the flattest curve yet. The peak does not occur until 180+ days.

Importantly, however, this does not result in signficantly lower fatalities as an outbreak occurs in the `group4b` *after the visit restriction is lifted*. 

This outcome was actually *much worse* than simply [restricting elderly visits](#2.-Restrict-Elderly-Visits). This is a counter-intuitive result and highlights the benefit of *allowing spread among the less vulnerable*.

#### 5. Quarantine ####
We can even mimick the impact of quarantines via the Quarantine object.

Here we show the impact of a 30-day quarantine for all groups in the Sim.

```python
from rknot.events import Quarantine

quarantine = Quarantine(name='all', start_tick=30, groups=[0,1,2,3,4], duration=30)
events_w_res = events_gated + [quarantine]

sim = Sim(groups=groups_gated, events=events_w_res, **params)
sim.run()

chart = Chart(sim, use_init_func=True)
chart.animate.to_html5_video()
```

In [9]:
if RUN:
    from rknot.events import Quarantine

    quarantine = Quarantine(name='all', start_tick=30, groups=[0,1,2,3,4], duration=30)
    events_w_res = events_gated + [quarantine]

    sim = Sim(groups=groups_gated, events=events_w_res, **params)
    sim.run()

    chart = Chart(sim, use_init_func=True)    
if SAVE_CHARTS:
    chart.animate.save('vids/baseus/quarantine.mp4', writer=FFMpegWriter(fps=10))

2020-10-17 00:34:48,172	INFO resource_spec.py:223 -- Starting Ray with 10.64 GiB memory available for workers and up to 5.34 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-10-17 00:34:48,600	INFO services.py:1191 -- View the Ray dashboard at localhost:8265


<video width='705' height='340'
       src='https://storage.googleapis.com/rknotvids/baseus/quarantine.mp4' controls>Video Failed to Load</video>

This results in:

|         |            |
| ------------- |:-------------:|
| Peak      |   1% |
| HIT      | 1%      |
| Fatalities | 0%      |

The pandemic never materializes in this simulation. The quarantine eradicates the virus swiftly. 

This is an interesting result. This obvioulsy did not occur in many places in the real world that took this approach. There are several possible reasons worth considering:

* Random Variation: if you run this simulation multiple times, you will note some sims where spread does occur.
* Scale: a larger simulation would increase the possibility that a very small number of subjects can continue to pass around the virus while it is muted in the broader population
* Adherence: real-world adherence to the implemented policies was much lower than this simulation suggests.
* Inconsistency: some areas implemented strict quarantines while others did not, and there was mixing among those populations.

### A Fulsome Scenario ###

We'll now create a more fulsome scenario encompassing many of the factors discussed.

Adjustments to prior scenarios include:

* add additional group with `mover='social'` in the 20-49 cohort. this cohort also flaunts social distancing guidelines as so its tmf in the SD object is `1.25`
* add susceptibility factors for most groups
* more fine-tuned capacity restrictions that gradually allow larger events over time
* restrict visits to the elderly for a full year

```python
### GROUPS ###
group1 = dict(
    name='0-19',
    n=2700,
    n_inf=0,
    ifr=0.00003,
    mover='local',
)
group2a = dict(
    name='20-49L',
    n=3900,
    n_inf=1,
    ifr=0.0002,
    mover='local',
    susf=0.85,    
)
group2b = dict(
    name='20-49S',
    n=200,
    n_inf=1,
    ifr=0.0002,
    mover='social',
    susf=0.85,    
)
group3 = dict(
    name='50-69',
    n=2300,
    n_inf=1,
    ifr=0.005,
    susf=.7,
    mover='local',
)
group4a = dict(
    name='70+',
    n=600,
    n_inf=0,
    ifr=0.042,
    mover='local',
    susf=0.9,    
)
group4b = dict(
    name='70+G',
    n=300,
    n_inf=0,
    ifr=0.0683,
    mover='local',
    box=[1,6,1,6],
    box_is_gated=True,
)
groups = [group1, group2a, group2b, group3, group4a, group4b]
params = {'dlevel': 'med', 'Ro':2.5, 'days': 365, 'imndur': 365, 'infdur': 14}

### EVENTS ###
from rknot.events import Event, Travel

school1 = Event(name='school1', xy=[25,42], start_tick=2, groups=[0], capacity=10, recurring=2)
school2 = Event(name='school2', xy=[78,82], start_tick=3, groups=[0], capacity=10, recurring=2)
game = Event(name='game', xy=[50,84], start_tick=6, groups=[0,1,2,3], capacity=100, recurring=14)
concert1 = Event(name='concert1', xy=[20,20], start_tick=7, groups=[1,2], capacity=50, recurring=14)                 
concert2 = Event(name='concert2', xy=[91,92], start_tick=21, groups=[1,2], capacity=50, recurring=14)
bar = Event(name='bar', xy=[17,24], start_tick=4, groups=[1,2], capacity=5, recurring=7)
bar2 = Event(name='bar2', xy=[87,13], start_tick=5, groups=[1,2], capacity=5, recurring=7)
bar3 = Event(name='bar3', xy=[52,89], start_tick=6, groups=[1,2,3], capacity=5, recurring=7)
church = Event(name='church', xy=[2,91], start_tick=7, groups=[3,4], capacity=20, recurring=7)
church2 = Event(name='church', xy=[2,3], start_tick=7, groups=[5], capacity=5, recurring=7)

visit = Travel(name='visit', xy=[1,1], start_tick=3, groups=[1,2,3], capacity=1, duration=1, recurring=1)

events = [
    school1, school2,
    game, concert1, concert2, bar, bar2, bar3, 
    church, church2, visit,
]

### RESTRICTIONS ###
small_gatherings = Restriction(
    name='small', start_tick=30, duration=30, criteria={'capacity': 5}
)
med_gatherings = Restriction(
    name='med', start_tick=60, duration=30, criteria={'capacity': 10}
)
large_gatherings = Restriction(
    name='large', start_tick=90, duration=30, criteria={'capacity': 50}
)
no_visits = Restriction(
    name='no_visits', start_tick=30, duration=365, criteria={'name': 'visit'}
)
sd = SD(name='all', tmfs=[.8, .8, 1.25, .7,.65,.5], groups=[0,1,2,3,4,5], start_tick=30, duration=120)
events_w_res = events + [small_gatherings, med_gatherings, large_gatherings, no_visits, sd]

sim = Sim(groups=groups, events=events_w_res, **params)
sim.run()

chart = Chart(sim, use_init_func=True)
chart.animate.to_html5_video()
```

In [10]:
### HIDDEN ###
if RUN:
    ### GROUPS ###
    group1 = dict(
        name='0-19',
        n=2700,
        n_inf=0,
        ifr=0.00003,
        mover='local',
    )
    group2a = dict(
        name='20-49L',
        n=3900,
        n_inf=1,
        ifr=0.0002,
        mover='local',
        susf=0.85,    
    )
    group2b = dict(
        name='20-49S',
        n=200,
        n_inf=1,
        ifr=0.0002,
        mover='social',
        susf=0.85,    
    )
    group3 = dict(
        name='50-69',
        n=2300,
        n_inf=1,
        ifr=0.005,
        susf=.7,
        mover='local',
    )
    group4a = dict(
        name='70+',
        n=600,
        n_inf=0,
        ifr=0.042,
        mover='local',
        susf=0.9,    
    )
    group4b = dict(
        name='70+G',
        n=300,
        n_inf=0,
        ifr=0.0683,
        mover='local',
        box=[1,6,1,6],
        box_is_gated=True,
    )
    groups = [group1, group2a, group2b, group3, group4a, group4b]
    params = {'dlevel': 'med', 'Ro':2.5, 'days': 365, 'imndur': 365, 'infdur': 14}

    ### EVENTS ###
    from rknot.events import Event, Travel

    school1 = Event(name='school1', xy=[25,42], start_tick=2, groups=[0], capacity=10, recurring=2)
    school2 = Event(name='school2', xy=[78,82], start_tick=3, groups=[0], capacity=10, recurring=2)
    game = Event(name='game', xy=[50,84], start_tick=6, groups=[0,1,2,3], capacity=100, recurring=14)
    concert1 = Event(name='concert1', xy=[20,20], start_tick=7, groups=[1,2], capacity=50, recurring=14)                 
    concert2 = Event(name='concert2', xy=[91,92], start_tick=21, groups=[1,2], capacity=50, recurring=14)
    bar = Event(name='bar', xy=[17,24], start_tick=4, groups=[1,2], capacity=5, recurring=7)
    bar2 = Event(name='bar2', xy=[87,13], start_tick=5, groups=[1,2], capacity=5, recurring=7)
    bar3 = Event(name='bar3', xy=[52,89], start_tick=6, groups=[1,2,3], capacity=5, recurring=7)
    church = Event(name='church', xy=[2,91], start_tick=7, groups=[3,4], capacity=20, recurring=7)
    church2 = Event(name='church', xy=[2,3], start_tick=7, groups=[5], capacity=5, recurring=7)

    visit = Travel(name='visit', xy=[1,1], start_tick=3, groups=[1,2,3], capacity=1, duration=1, recurring=1)

    events = [
        school1, school2,
        game, concert1, concert2, bar, bar2, bar3, 
        church, church2, visit,
    ]

    ### RESTRICTIONS ###
    small_gatherings = Restriction(
        name='small', start_tick=30, duration=30, criteria={'capacity': 5}
    )
    med_gatherings = Restriction(
        name='med', start_tick=60, duration=30, criteria={'capacity': 10}
    )
    large_gatherings = Restriction(
        name='large', start_tick=90, duration=30, criteria={'capacity': 50}
    )
    no_visits = Restriction(
        name='no_visits', start_tick=30, duration=365, criteria={'name': 'visit'}
    )
    sd = SD(name='all', tmfs=[.8, .8, 1.25, .7,.65,.5], groups=[0,1,2,3,4,5], start_tick=30, duration=120)
    events_w_res = events + [small_gatherings, med_gatherings, large_gatherings, no_visits, sd]

    sim = Sim(groups=groups, events=events_w_res, **params)
    sim.run()

    chart = Chart(sim, use_init_func=True)
if SAVE_CHARTS:
    chart.animate.save('vids/baseus/fulsome.mp4', writer=FFMpegWriter(fps=10))

2020-10-17 00:37:13,724	INFO resource_spec.py:223 -- Starting Ray with 10.3 GiB memory available for workers and up to 5.17 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-10-17 00:37:14,151	INFO services.py:1191 -- View the Ray dashboard at localhost:8265


<video width='705' height='340'
       src='https://storage.googleapis.com/rknotvids/baseus/fulsome.mp4' controls>Video Failed to Load</video>

This results in:

|         |            |
| ------------- |:-------------:|
| Peak      |   14% |
| HIT      | 35%      |
| Fatalities | 0.23%      |

The virus barely survives for the first 6-months of the outbreak, then as per other scenarios, once restrictions are lifted an outbreak occurs. Still, when the outbreak does occur, it is characterized by one of the lowest peaks and HITs in our analysis (due in part to the pre-immmunity of some of the groups).

And it achieves the lowest fatality rate of the group, mainly by restricting access to the elderly population for the duration of the pandemic and ensuring an outbreak never occurs in that region.

In [7]:
import os
dir_ = os.getcwd()
dir_docs = dir_ + '/docs/docs'
filename = '/hit.ipynb'

from shutil import copyfile
copyfile(dir_ + filename, dir_docs + filename)

'/Users/spindicate/Documents/programming/rknot/docs/docs/hit.ipynb'